# NAME 1: TAMARALAYEFA AYEMIDI TIMIYAN #NAME 2: KUMAR RAHUL


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
Xl = pd.read_csv('/content/gdrive/MyDrive/Xl.csv',header=None)
Xs = pd.read_csv('/content/gdrive/MyDrive/Xs.csv',header=None)
R = pd.read_csv('/content/gdrive/MyDrive/y.csv',header = None)

In [4]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [5]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [6]:
np.shape(t_test_end)

(552,)

In [7]:
t_test_end[551]

731

In [8]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=0.003)
    model.compile(optimizer=optimizer,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=2)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=50,verbose=1,callbacks=[early_stop])
    preds.append(model.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 32ms/step - loss: 0.0030 - val_loss: 0.0217
Epoch 19/50
1/1 [==============================] - 0s 38ms/step - loss: 0.0029 - val_loss: 0.0211
Epoch 20/50
1/1 [==============================] - 0s 33ms/step - loss: 0.0028 - val_loss: 0.0205
Epoch 21/50
1/1 [==============================] - 0s 33ms/step - loss: 0.0027 - val_loss: 0.0200
Epoch 22/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0026 - val_loss: 0.0196
Epoch 23/50
1/1 [==============================] - 0s 33ms/step - loss: 0.0025 - val_loss: 0.0193
Epoch 24/50
1/1 [==============================] - 0s 31ms/step - loss: 0.0024 - val_loss: 0.0191
Epoch 25/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0022 - val_loss: 0.0188
Epoch 26/50
1/1 [==============================] - 0s 40ms/step - loss: 0.0021 - val_loss: 0.0187
Epoch 27/50
1/1 [==============================] - 0s 44ms/step - loss: 0.0019 

In [9]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

### Calculate R^2

In [10]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.290955
dtype: float64

### A good R^2 is expected to be between 0 to 0.05.